In [1]:
! pip install huggingface_hub transformers datasets peft trl bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.7 MB/s eta 0:00:00


In [2]:
import re
import torch
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from random import randint
from huggingface_hub import login
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, pipeline
from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig, AutoPeftModelForCausalLM

2024-06-17 02:55:06.434466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 02:55:06.434575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 02:55:06.565435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
login(
  token = "your_token",
  add_to_git_credential=True
)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


Iam using SFTTrainer which accepts input data in some particular formt. You can refer to the formats here - https://huggingface.co/docs/trl/en/sft_trainer#dataset-format-support. Iam using the conversational format which will be something like - 
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}

In [4]:
def extract_schema_and_question(text):
    schema_pattern = r'\[INST\] Here is a database schema:(.*?)Please write me a SQL statement that answers the following question:'
    question_pattern = r'Please write me a SQL statement that answers the following question:(.*?)\[/INST\]'

    schema_match = re.search(schema_pattern, text, re.DOTALL)
    question_match = re.search(question_pattern, text, re.DOTALL)

    if schema_match and question_match:
        schema = schema_match.group(1).strip()
        question = question_match.group(1).strip()
        return schema, question
    else:
        return None, None

In [5]:
system_prompt = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_instruction(sample):
    schema, question = extract_schema_and_question(sample["input"])
    if schema and question:
        return {
        "messages": [
              {"role": "system", "content": system_prompt.format(schema = schema)},
              {"role": "user", "content": question},
              {"role": "assistant", "content": sample["output"]}
            ]
          }  
    else:
        return None

Iam using the dataset available at https://huggingface.co/datasets/lamini/bird_text_to_sql which contains the input with the schema of the database and query, and the output as the SQL query. Since I donot have access to any paid or larger GPUs, Iam training the model only on 1000 samples, which can be considered a decent size of samples to perform finetuning. 

In [6]:
dataset = load_dataset("lamini/bird_text_to_sql")

combined_dataset = concatenate_datasets([dataset["train"], dataset["dev"]])

processed_dataset = combined_dataset.map(lambda x: create_instruction(x), remove_columns=combined_dataset.column_names)

processed_dataset = processed_dataset.shuffle()
processed_dataset = processed_dataset.train_test_split(test_size = 0.15)

print(processed_dataset["train"][69])

train_sampled = processed_dataset["train"].select(range(1000))
test_sampled = processed_dataset["test"].select(range(500))

train_sampled.to_json("train_dataset.json", orient="records")
test_sampled.to_json("test_dataset.json", orient="records")

Generating train split:   0%|          | 0/9428 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1534 [00:00<?, ? examples/s]

{'messages': [{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nIngredient :\ningredient_id [ INTEGER ] primary_key\ncategory [ TEXT ]\nname [ TEXT ]\nplural [ TEXT ]\n\nRecipe :\nrecipe_id [ INTEGER ] primary_key\ntitle [ TEXT ]\nsubtitle [ TEXT ]\nservings [ INTEGER ]\nyield_unit [ TEXT ]\nprep_min [ INTEGER ]\ncook_min [ INTEGER ]\nstnd_min [ INTEGER ]\nsource [ TEXT ]\nintro [ TEXT ]\ndirections [ TEXT ]\n\nNutrition :\nrecipe_id [ INTEGER ] primary_key Nutrition.recipe_id = Recipe.recipe_id\nprotein [ REAL ]\ncarbo [ REAL ]\nalcohol [ REAL ]\ntotal_fat [ REAL ]\nsat_fat [ REAL ]\ncholestrl [ REAL ]\nsodium [ REAL ]\niron [ REAL ]\nvitamin_c [ REAL ]\nvitamin_a [ REAL ]\nfiber [ REAL ]\npcnt_cal_carb [ REAL ]\npcnt_cal_fat [ REAL ]\npcnt_cal_prot [ REAL ]\ncalories [ REAL ]\n\nQuantity :\nquantity_id [ INTEGER ] primary_key\nrecipe_id [ INTEGER ] Quantity.recipe_id 

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1347585

In [7]:
train_dataset = load_dataset("json", data_files = "train_dataset.json", split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

Since I donot have access to any paid GPU services, Iam training the model on free GPU P100 instance provided by Kaggle, which is obviously not sufficient to train all 7 billion parameters of the model. So, I'm using LORA and quantization techniques in the training process. If you have access to larger GPUs, go ahead and experiment by increasing the lora_rank, set bf16 to True, increase the size of train data, and also you can include a technique called FLASH-ATTENTION which accelerates the training upto 3x. If you have access to google collabs paid version and A100 GPU, you can experiment with above parameters.

In [8]:
model_id = "google/codegemma-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'
model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [9]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=128,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
args = TrainingArguments(
    output_dir="code-gemma-7b-text-to-sql",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=30,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=False,
    fp16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.05,
    lr_scheduler_type="linear",
    weight_decay=0.01
)

In [11]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:181: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use 

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = model.num_parameters()
    for _, param in model.named_parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

trainable params: 51380224 || all params: 8589067264 || trainable%: 0.5982049321624686


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,36.730400
60,1.789800
90,0.853200
120,0.587600
150,0.494900
180,0.442100
210,0.440400
240,0.426300
270,0.350100
300,0.353700


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=402, training_loss=3.250924762208663, metrics={'train_runtime': 6503.9821, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.062, 'total_flos': 3.851927038722048e+16, 'train_loss': 3.250924762208663, 'epoch': 1.0})

As you can see, training the model only ofr 1 epoch, that too with LORA and Quantization tokk me almost 2 hours. I tried increasing the Lora-rank, batch_size, and also max_seq_lenght parameters, but obviously it threw out of memory exception. As i mentioned earlier, If you have access to google collabs paid version and A100 GPU, you can increase the train data, and also train it for more epochs to get better results.

In [14]:
trainer.save_model()
trainer.model.push_to_hub("code-gemma-7b-finetuned-text-to-sql")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/6.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nishanth7803/code-gemma-7b-finetuned-text-to-sql/commit/5f2f5de3ebb294cf09cb1e978e054715654ffdc7', commit_message='Upload model', commit_description='', oid='5f2f5de3ebb294cf09cb1e978e054715654ffdc7', pr_url=None, pr_revision=None, pr_num=None)

In [15]:
del model
del trainer

In [21]:
torch.cuda.empty_cache()

In [22]:
peft_model_id = "/kaggle/working/code-gemma-7b-text-to-sql"
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/opt/conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1365: UserWarning: Current model requires 7168 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCaus

In [23]:
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# Test on sample 
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

print(f"Schema:\n{eval_dataset[rand_idx]['messages'][0]['content']}")
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Schema:
You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
Examination :
ID [ INTEGER ] Examination.ID = Patient.ID
Examination Date [ DATE ]
aCL IgG [ REAL ]
aCL IgM [ REAL ]
ANA [ INTEGER ]
ANA Pattern [ TEXT ]
aCL IgA [ INTEGER ]
Diagnosis [ TEXT ]
KCT [ TEXT ]
RVVT [ TEXT ]
LAC [ TEXT ]
Symptoms [ TEXT ]
Thrombosis [ INTEGER ]

Patient :
ID [ INTEGER ] primary_key
SEX [ TEXT ]
Birthday [ DATE ]
Description [ DATE ]
First Date [ DATE ]
Admission [ TEXT ]
Diagnosis [ TEXT ]

Laboratory :
ID [ INTEGER ] Laboratory.ID = Patient.ID
Date [ DATE ]
GOT [ INTEGER ]
GPT [ INTEGER ]
LDH [ INTEGER ]
ALP [ INTEGER ]
TP [ REAL ]
ALB [ REAL ]
UA [ REAL ]
UN [ INTEGER ]
CRE [ REAL ]
T-BIL [ REAL ]
T-CHO [ INTEGER ]
TG [ INTEGER ]
CPK [ INTEGER ]
GLU [ INTEGER ]
WBC [ REAL ]
RBC [ REAL ]
HGB [ REAL ]
HCT [ REAL ]
PLT [ INTEGER ]
PT [ REAL ]
APTT [ INTEGER ]
FG [ REAL ]
PIC [ INTEGER ]
TAT [ INTEGER

You can see that after training for only 1 epoch, and only on 1000 samples, our model outputs not very perfect result, but a decent resut I would say. Obviously more improvements can be done, so go ahead and try experimenting with different parameters. If you think any other improvements can be made, feel free to drop a mail to nishanth.annamdevula7803@gmail.com

HAPPY BUILDING :)